<a href="https://colab.research.google.com/github/dyoni2/jiyeon/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**정형 데이터와 비정형 데이터**

정형 데이터: CSV 파일처럼 구조를 가지고 있는 데이터

비정형 데이터: 텍스트 데이터, 사진, 디지털 음악 등 구조를 가지고 있지 않은 데이터 <-6장

## 앙상블 학습

정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘

대다수의 앙상블 학습 알고리즘은 결정 트리를 기반으로 함

# 랜덤 포레스트

앙상블 학습 중 가장 유명하고 안정적인 성능을 제공

1. 훈련하기 위한 데이터를 랜덤하게 만든다
- 입력한 훈련 데이터에서 랜덤하게 샘플을 추출함
- 중복된 샘플을 추출할 수 있음
- 이렇게 만들어진 샘플을 **부트스트랩 샘플**이라 함
- 기본적으로 훈련 세트의 크기와 동일하게 설정됨

2. 부트스트랩 샘플로 결정 트리를 훈련
- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음
- RandomForestClassifier 분류 모델은 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택
- RandomForestRegressor 회귀 모델은 전체 특성을 사용

3. 정해진 수 만큼 위 과정을 반복

랜덤하게 선택한 샘플과 특성을 사용하기 떄문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.

랜덤 포레스트는 누락된 값이 있어도 처리가 가능하다는 장점이 있다.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


이 예제는 매우 간단하고 특성이 많지 않아서 그리드 서치를 사용하더라도 하이퍼 파라미터 튜닝의 결과가 크게 나아지지 않음

특성 중요도 출력

In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 떄문에 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 준다.

따라서 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됨

**oob_score**

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차검증에서 얻은 점수와 매우 비슷한 점수를 얻음

# 엑스트라 트리

랜덤 포레스트와 비슷하게 동작
- 부트스트랩 샘플을 사용하지 않고, 각 결정 트리를 만들 때 전체 훈려ㅕㄴ 세트를 사용
-노드를 분할할 때 가장 좋은 분할을 찾지 않고 랜덤으로 분할함

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# Bagging과 Boosting

배깅:

부스팅 : 모델을 순차적으로 여러개 만들되 이전 모델을 보완한 새로운 모델을 만들고 최종적으로 생성된 모든 모델을 하나로 합침

# 그레이디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식

사이킷런에서는 기본적으로 깊이가 3인 결정 트리 100개를 사용

깊이가 얕은 결정 트리를 사용하므로 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음

경사 하강법을 사용하여 트리를 앙상블에 추가한다

분류에서는 로지스틱 손실 함수를 사용

회귀에서는 평균 제곱 오차 함수를 사용

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


과대적합을 적절히 억제하고 있음

In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그래디언트 부스팅

정형 데이터를 다루는 머신러닝 알고리즘 중에서 가장 인기가 높은 알고리즘

입력 특성을 256개의 구간으로 나누므로 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음

256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용함

즉, 어떤 샘플에 누락된 값이 있다면 뗴어놓은 구간의 값을 대신 사용함

따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음

히스토그램: 데이터를 일정한 구간으로 쪼개서 막대그래프로 표현

입력 데이터의 각 특성을 256개의 구간으로 나누어 256개의 값으로 변환

In [11]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


훈련 세트에 대해서도 특성 중요도를 계산할 수 있음

In [12]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

**XGBoost**

사이킷런의 cross_validate()함수 사용 가

In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


**LightGBM**



In [16]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


**실습 5.**

WINE 데이터 세트를 이용하여
- 결정트리
- 랜덤포레스트
- 엑스트라 트리
- 그래디언트 부스팅
- 히스토그램기반 그래디언트 부스팅

모델을 만들고 최적화

WINE 데이터 세트(이진분류)
https://archive.ics.uci.edu/dataset/186/wine+quality

In [8]:
# 결정 트리
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

from sklearn.tree import DecisionTreeClassifier

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_scaled, train_target)

print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

0.7808350971714451
0.7776923076923077
0.996921300750433
0.8592307692307692


In [7]:
# 랜덤포레스트
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [9]:
#엑스트라 트리
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print(et.feature_importances_)

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


In [11]:
#그래디언트 부스팅
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015
0.9464595437171814 0.8780082549788999


In [12]:
#히스토그램 기반 그래디언트 부스팅
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

hgb.score(test_input, test_target)

0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


0.8723076923076923